<div style="
    font-family: 'Old English Text MT', 'Goudy Bookletter 1911', serif;
    font-size: 38px;
    font-weight: bold;
    letter-spacing: 2px;
    color:rgb(197, 56, 37);
    text-align: center;
    margin-bottom: 20px;
">
Esercitazione IV
</div>

<div style="
    font-family: 'Goudy Bookletter 1911', 'Old English Text MT', 'Times New Roman', serif;
    font-size: 18px;
    letter-spacing: 1.5px;
    line-height: 1.4;
    color:rgb(244, 105, 46); /* Goldenrod */
    text-align: center;
">
Addestra un LSTM a generare testo nello stile di Shakespeare, prevedendo un carattere alla volta.
</div>

In [1]:
import pyfiglet

ascii_text = pyfiglet.figlet_format("Shakespeare LSTM")
print(ascii_text)

 ____  _           _                                       
/ ___|| |__   __ _| | _____  ___ _ __   ___  __ _ _ __ ___ 
\___ \| '_ \ / _` | |/ / _ \/ __| '_ \ / _ \/ _` | '__/ _ \
 ___) | | | | (_| |   <  __/\__ \ |_) |  __/ (_| | | |  __/
|____/|_| |_|\__,_|_|\_\___||___/ .__/ \___|\__,_|_|  \___|
                                |_|                        
 _     ____ _____ __  __ 
| |   / ___|_   _|  \/  |
| |   \___ \ | | | |\/| |
| |___ ___) || | | |  | |
|_____|____/ |_| |_|  |_|
                         



## Setup

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import requests
import re
from torch.utils.data import Dataset, DataLoader

Per addestrare la rete LSTM a generare testo a livello di carattere è necessario disporre di un corpus sufficientemente lungo, però lo stile di Shakespeare è ben definito e facilmente riconoscibile.

Il testo originale contiene oltre 5.000.000 di caratteri. Tuttavia, ho deciso di utilizzare solo i primi 50_000 caratteri per ridurre i tempi di addestramento e permettermi di sperimentare più velocemente diverse configurazioni del modello.

In [ ]:
import re

print("Loading Shakespeare from local file...")

file_path = "shakespeare.txt"

with open(file_path, "r", encoding="utf-8") as f:
    text = f.read()

print("Raw characters:", len(text))

text = re.sub(r'\s+', ' ', text)
text = text[:50_000]

print("Characters used:", len(text))

Loading Shakespeare from local file...
Raw characters: 5376400
Characters used: 50000


## Costruzione del vocabolario

Inizialmente, ho individuato tutti i caratteri unici presenti nel corpus e questo insieme di caratteri costituirà il vocabolario del modello. 

Successivamente, si costruisce una corrispondenza tra ogni carattere e un numero intero e si crea anche la mappatura inversa, da numero a carattere.

Infine, si sostituisce ogni carattere con il suo indice corrispondente nel vocabolario. 

In [52]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(f"Unique characters: {vocab_size}\nSample: {chars[:20]}...")

char_to_idx = {ch: i for i, ch in enumerate(chars)}
idx_to_char = dict(enumerate(chars))

encoded = np.array([char_to_idx[c] for c in text])
print(f"Encoded text shape: {encoded.shape}")
print(f"First 20 encoded values: {encoded[:20]}")

Unique characters: 71
Sample: [' ', '!', '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?']...
Encoded text shape: (50000,)
First 20 encoded values: [ 0  8  0 25 59 56 54  0 47 42 50 59 46 60 61  0 44 59 46 42]


## Dataset

Ora, consideriamo il testo codificato (cioè la sequenza di numeri ottenuta), e lo dividiamo in sequenze di lunghezza fissa, definite dal parametro `SEQ_LENGTH`.

`x` è una sequenza di 100 caratteri (convertiti in numeri)

`y` è la stessa sequenza, spostata di una posizione in avanti.

Il DataLoader viene utilizzato per organizzare questi esempi in batch, gruppi di `64` sequenze alla volta.  Inoltre, ho aggiunto anche l’opzione `shuffle=True`.
L'insieme ordinato e pronto di sequenze verrà dato in input al modello `LSTM` nella fase di training.

In [53]:
SEQ_LENGTH = 100

class CharDataset(Dataset):
    def __init__(self, data, seq_len):
        self.data = data
        self.seq_len = seq_len

    def __len__(self):
        return len(self.data) - self.seq_len

    def __getitem__(self, idx):
        x = torch.tensor(self.data[idx:idx+self.seq_len], dtype=torch.long)
        y = torch.tensor(self.data[idx+1:idx+self.seq_len+1], dtype=torch.long)
        return x, y

dataset = CharDataset(encoded, SEQ_LENGTH)
loader = DataLoader(dataset, batch_size=64, shuffle=True, drop_last=True)
print(f"Total sequences: {len(dataset)} | DataLoader ready")

Total sequences: 49900 | DataLoader ready


## Architettura del modello

Il modello è composto da tre parti principali:

1. Embedding layer `(nn.Embedding)`

Il layer di embedding trasforma ogni indice in un vettore denso di dimensione `embed_size` (in questo caso `256`). Questo passaggio serve a fornire al modello una rappresentazione più significativa dei caratteri.

2. LSTM multilayer `(nn.LSTM)`

`hidden_size = 512` è la dimensione dello stato nascosto

`num_layers = 3` LSTM è composta da 3 strati sovrapposti

`dropout = 0.2` spegne casualmente il 20% delle connessioni tra gli strati 

Ho aggiunto anche la possibilità di usare una LSTM bidirezionale, però in questo caso è disattivata perché grazie alla memoria interna, il modello è abbastanza in grado di cogliere dipendenze tra caratteri lontani all’interno del testo (anche punteggiature e strutture sintattiche).

3. Fully connected layer `(nn.Linear)`

Ogni valore di questo vettore finale rappresenta uno score per ogni possibile carattere successivo. Durante il training, questi score verranno confrontati con il carattere reale usando la funzione di loss.

Utilizzo `CrossEntropyLoss` come funzione di perdita e `Adam` come ottimizzatore, con `learning rate = 0.001`.

In [54]:
class CharLSTM(nn.Module):
    def __init__(self, vocab_size, embed_size=256, hidden_size=512, num_layers=3, dropout=0.2, bidirectional=False):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.bidirectional = bidirectional

        self.embed = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.LSTM(
            embed_size, hidden_size, num_layers,
            batch_first=True,
            dropout=dropout if num_layers>1 else 0,
            bidirectional=bidirectional
        )
        self.fc = nn.Linear(hidden_size * (2 if bidirectional else 1), vocab_size)

    def forward(self, x, hidden=None):
        x = self.embed(x)
        out, hidden = self.lstm(x, hidden)
        out = self.fc(out)
        return out, hidden

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CharLSTM(vocab_size, hidden_size=512, num_layers=3, dropout=0.2, bidirectional=False).to(device)
print(f"Using device: {device}")
print(model)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

Using device: cuda
CharLSTM(
  (embed): Embedding(71, 256)
  (lstm): LSTM(256, 512, num_layers=3, batch_first=True, dropout=0.2)
  (fc): Linear(in_features=512, out_features=71, bias=True)
)


## Addestramento

In [55]:
EPOCHS = 5 
print("Training model...")
for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    hidden = None

    for i, (X_batch, y_batch) in enumerate(loader):
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()

        outputs, hidden = model(X_batch, hidden)

        if hidden is not None:
            hidden = (hidden[0].detach(), hidden[1].detach())

        loss = criterion(outputs.reshape(-1, vocab_size), y_batch.reshape(-1))
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), 5)
        optimizer.step()
        total_loss += loss.item()

        if i % 100 == 0:
            print(f"  Batch {i}/{len(loader)} | Loss: {loss.item():.4f}")

    print(f"=== Epoch {epoch+1}/{EPOCHS} finished | Avg Loss: {total_loss/len(loader):.4f}")

Training model...
  Batch 0/779 | Loss: 4.2701
  Batch 100/779 | Loss: 2.3172
  Batch 200/779 | Loss: 1.9220
  Batch 300/779 | Loss: 1.6269
  Batch 400/779 | Loss: 1.4872
  Batch 500/779 | Loss: 1.3491
  Batch 600/779 | Loss: 1.2154
  Batch 700/779 | Loss: 1.0873
=== Epoch 1/5 finished | Avg Loss: 1.6571
  Batch 0/779 | Loss: 0.9664
  Batch 100/779 | Loss: 0.8310
  Batch 200/779 | Loss: 0.7166
  Batch 300/779 | Loss: 0.6037
  Batch 400/779 | Loss: 0.5063
  Batch 500/779 | Loss: 0.4632
  Batch 600/779 | Loss: 0.4121
  Batch 700/779 | Loss: 0.3642
=== Epoch 2/5 finished | Avg Loss: 0.5691
  Batch 0/779 | Loss: 0.3265
  Batch 100/779 | Loss: 0.3119
  Batch 200/779 | Loss: 0.2986
  Batch 300/779 | Loss: 0.2852
  Batch 400/779 | Loss: 0.2684
  Batch 500/779 | Loss: 0.2733
  Batch 600/779 | Loss: 0.2674
  Batch 700/779 | Loss: 0.2647
=== Epoch 3/5 finished | Avg Loss: 0.2872
  Batch 0/779 | Loss: 0.2382
  Batch 100/779 | Loss: 0.2417
  Batch 200/779 | Loss: 0.2268
  Batch 300/779 | Loss: 0.2

## Generazione del testo 

La funzione prende in input:

`start_text`: una sequenza iniziale di caratteri da cui partire.

`length`: il numero di caratteri che vogliamo generare.

`temperature`: un parametro che regola il randomness (creatività) del testo.

La `temperature` viene applicata ai `logits`, dove valori più bassi rendono le predizioni più ripetitive.

In [56]:
def generate_text(model, start_text="To be ", length=500, temperature=1.0):
    model.eval()
    input_seq = torch.tensor([char_to_idx[c] for c in start_text], dtype=torch.long).unsqueeze(0).to(device)
    hidden = None
    result = start_text

    with torch.no_grad():
        for _ in range(length):
            out, hidden = model(input_seq, hidden)
            logits = out[:, -1, :] / temperature
            probs = torch.softmax(logits, dim=1).cpu().numpy().squeeze()
            next_idx = np.random.choice(len(probs), p=probs)
            result += idx_to_char[next_idx]
            input_seq = torch.tensor([[next_idx]], dtype=torch.long).to(device)

    return result

## Perplexity

La funzione perplexity serve solo come strumento di valutazione aggiuntivo per quantificare numericamente quanto il modello sia bravo a prevedere i caratteri, senza dover leggere manualmente il testo generato. 

Prende in input:
`text_sample`: un frammento di testo da valutare.

Pertanto, per valori bassi, vuol dire che il modello è sicuro e il testo è facile da prevedere.

In [57]:
def perplexity(model, text_sample):
    model.eval()
    indices = torch.tensor([char_to_idx[c] for c in text_sample if c in char_to_idx]).to(device)
    input_seq = indices[:-1].unsqueeze(0)
    target = indices[1:]

    with torch.no_grad():
        outputs, _ = model(input_seq)
        outputs = outputs.squeeze(0)
        loss = nn.functional.cross_entropy(outputs, target)
    return torch.exp(loss).item()

## Esempi

In [58]:
generate_text(model, "To be ", 500, temperature=0.5)

'To be the painter and hath stelled, Thy beauty’s form in table of my heart, My body is the frame wherein ’tis held, And perspective it is best painter’s art. For through the painter must you see his skill, To find where your true image pictured lies, Which in my bosom’s shop is hanging still, That hath his windows glazed with thine eyes: Now see what good turns eyes for eyes have done, Mine eyes have drawn thy shape, and thine for me Are windows to my breast, where-through the sun Delights to peep, to'

In [59]:
generate_text(model, "To be ", 500, temperature=0.5)

'To be the basest clouds to ride, With ugly rack on his celestial face, And from the forlorn world his visage hide Stealing unseen to west with this disgrace: Even so my sun one early morn did shine, With all triumphant splendour on my brow, But out alack, he was but one hour mine, The region cloud hath masked him from me now. Yet him for this, my love no whit disdaineth, Suns of the world may stain, when heaven’s sun staineth. 34 Why didst thou promise such a beauteous day, And make me travel forth wi'

In [60]:
generate_text(model, "To be or ", 500, temperature=0.5)

'To be or all the all of me. 32 If thou survive my well-contented day, When that churl death my bones with dust shall cover And shalt by fortune once more re-survey These poor rude lines of thy deceased lover: Compare them with the bett’ring of the time, And though they be outstripped by every pen, Reserve them for my love, not for their rhyme, Exceeded by the height of happier men. O then vouchsafe me but this loving thought, ’Had my friend’s Muse grown with this growing age, A dearer birth than this his'

In [64]:
generate_text(model, "Artificial intelligence will rule the ", 300, temperature=0.5)

'Artificial intelligence will rule the world’s due, by the grave and thee. 2 When forty winters shall besiege thy brow, And dig deep trenches in thy beauty’s field, Thy youth’s proud livery so gazed on now, Will be a tattered weed of small worth held: Then being asked, where all thy beauty lies, Where all the treasure of thy lusty days; '

## Valutazione

In [63]:
sample_text = generate_text(model, start_text="To be ", length=200, temperature=0.5)

pp = perplexity(model, sample_text)
print(f"Perplexity of the generated text: {pp:.2f}")

Perplexity of the generated text: 1.09
